In [3]:
import pandas as pd
import re
from pathlib import Path

base = Path("results")

# Arquivos (ajustado conforme sua foto)
paths = {
    ("Kafka",    "Script custom (console-producer)", "custom"): base / "producer_custom.txt",
    ("Kafka",    "Ferramenta nativa (perf-test)",    "native"): base / "producer_native.txt",
    ("Redpanda", "rpk topic produce",                "tempo"):  base / "producer_redpanda_rpk.txt",
    ("Pulsar",   "pulsar-perf produce",              "tempo"):  base / "producer_pulsar_perf.txt",  # <- aqui estava o problema
}

def parse_custom_or_tempo(path: Path):
    text = path.read_text(errors="ignore")

    # Formato do Kafka custom:
    # Sent 20000 messages in 12s
    m = re.findall(r"Sent\s+(\d+)\s+messages\s+in\s+([\d\.]+)s", text)
    if m:
        num = int(m[-1][0])
        secs = float(m[-1][1])
        thr = num / secs if secs > 0 else float("nan")
        return num, secs, thr

    # Formato do Redpanda/Pulsar (scripts):
    # Tempo total para enviar 20000 mensagens: 12s
    m = re.findall(r"Tempo\s+total\s+para\s+enviar\s+(\d+)\s+mensagens:\s*([\d\.]+)s", text, flags=re.IGNORECASE)
    if m:
        num = int(m[-1][0])
        secs = float(m[-1][1])
        thr = num / secs if secs > 0 else float("nan")
        return num, secs, thr

    raise ValueError(f"Não achei padrão de tempo/mensagens no arquivo: {path.name}")

def parse_native_kafka(path: Path):
    text = path.read_text(errors="ignore")
    # Formato típico do kafka-producer-perf-test / perf:
    # 20000 records sent, 5000.0 records/sec
    m = re.findall(r"(\d+)\s+records sent.*?([\d\.]+)\s+records/sec", text, flags=re.DOTALL)
    if not m:
        raise ValueError(f"Não encontrei linha 'records sent ... records/sec' em: {path.name}")
    num = int(m[-1][0])
    thr = float(m[-1][1])
    secs = num / thr if thr > 0 else float("nan")
    return num, secs, thr

rows = []
missing = []

for (tech, approach, mode), path in paths.items():
    if not path.exists():
        missing.append(str(path))
        continue

    if mode == "native":
        num, secs, thr = parse_native_kafka(path)
    else:
        num, secs, thr = parse_custom_or_tempo(path)

    rows.append({
        "Tecnologia": tech,
        "Abordagem": approach,
        "# Mensagens": num,
        "Tempo (s)": round(secs, 2),
        "Throughput (msg/s)": round(thr, 2),
        "Arquivo": path.name,
    })

if missing:
    raise FileNotFoundError("Arquivos não encontrados em results/:\n- " + "\n- ".join(missing))

df = pd.DataFrame(rows).sort_values(["Tecnologia", "Abordagem"]).reset_index(drop=True)
display(df)

latex = df.to_latex(
    index=False,
    caption="Resultados dos testes de produção de mensagens (Kafka, Redpanda e Pulsar).",
    label="tab:resultados-brokers"
)
print(latex)


,Tecnologia,Abordagem,# Mensagens,Tempo (s),Throughput (msg/s),Arquivo
0,Kafka,Ferramenta nativa (perf-test),20000,0.72,27662.50,producer_native.txt
1,Kafka,Script custom (console-producer),20000,87.00,229.89,producer_custom.txt
2,Pulsar,pulsar-perf produce,20000,12.00,1666.67,producer_pulsar_perf.txt
3,Redpanda,rpk topic produce,20000,49.00,408.16,producer_redpanda_rpk.txt


\begin{table}
\caption{Resultados dos testes de produção de mensagens (Kafka, Redpanda e Pulsar).}
\label{tab:resultados-brokers}
\begin{tabular}{llrrrl}
\toprule
Tecnologia & Abordagem & # Mensagens & Tempo (s) & Throughput (msg/s) & Arquivo \\
\midrule
Kafka & Ferramenta nativa (perf-test) & 20000 & 0.720000 & 27662.500000 & producer_native.txt \\
Kafka & Script custom (console-producer) & 20000 & 87.000000 & 229.890000 & producer_custom.txt \\
Pulsar & pulsar-perf produce & 20000 & 12.000000 & 1666.670000 & producer_pulsar_perf.txt \\
Redpanda & rpk topic produce & 20000 & 49.000000 & 408.160000 & producer_redpanda_rpk.txt \\
\bottomrule
\end{tabular}
\end{table}

